## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre:Alvaro
- Apellido:Ticona Yujra
- Legajo:155561-3

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [4]:
#se importa el dataset
london = pd.read_csv("TP2_airbnb_london.csv")
#se corrobora uqe hata cargo correctamente
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [5]:
#se imprime dimensiones del dataset
london.shape

(51827, 9)

In [6]:
#se elimina la variable y se guarda en london.
london = london.drop(columns=['Unnamed: 0'])

##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [7]:
#se observa que no haya nulos por columnas
london.isnull().sum()

neighbourhood                     0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [8]:
#tipo de dato de "neighbourhood"
type("neighbourhood")

str

In [9]:
#tipo de dato de "room_type"
type("room_type")

str

##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [10]:
## Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
neigh_dummies = pd.get_dummies(london["neighbourhood"])

Imprimir las dimensiones y primeras lineas de estas dummies

In [11]:
#se imprime las dimensiones
neigh_dummies.shape

(51827, 33)

In [12]:
#se imprime primeras lineas de la dummies
neigh_dummies.head()

,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
## Genero las dummies para la variable room_type, dentro de la variable room_dummies.
room_dummies =pd.get_dummies(london["room_type"])

Imprimir las dimensiones y primeras lineas de estas dummies

In [14]:
#se imprime las dimensiones.
room_dummies.shape

(51827, 3)

In [15]:
#se imprime las primeras lineas.
room_dummies.head()

,Entire home/apt,Private room,Shared room
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


Joinear las variables dummies al dataset



In [16]:
#se concatena las variables dummies al sataset.
london = pd.concat([london,room_dummies,neigh_dummies],axis=1)

##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [17]:
y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

In [18]:
#por cuestiones de costo computacional de usa las primeras 5000 muestras.
x = x.iloc[0:5000,:]
y=y[0:5000]

In [19]:
y.shape

(5000, 1)

In [20]:
x.shape

(5000, 41)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [21]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.30, random_state=42)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [22]:
scaler = preprocessing.StandardScaler().fit(xtrain)

In [23]:
xtrain_scal = scaler.transform(xtrain)

In [25]:
xtest_scal = scaler.transform(xtest)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [26]:
#se define el modelo lineal
modelo_lineal= linear_model.LinearRegression()

In [27]:
#entreno el modelo
modelo_lineal.fit(xtrain_scal, ytrain)

LinearRegression()

In [28]:
#se reaiza las predicciones.
ypred1 = modelo_lineal.predict(xtest_scal)

In [29]:
#calculo el mean_squared_error.
np.sqrt(mean_squared_error(ytest, ypred1))

36.261774226165414

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [30]:
svreg = SVR()

In [31]:
param_svreg = {'kernel':('linear', 'rbf'), 'C':[1, 10, 21 ], 'gamma':[ 3,30,40]}

In [32]:
regressor_svr = GridSearchCV(svreg, param_svreg, refit=True ,cv=5, verbose=True)

In [33]:
#se entrena el modelo
regressor_svr.fit(xtrain_scal, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 10, 21], 'gamma': [3, 30, 40],
                         'kernel': ('linear', 'rbf')},
             verbose=True)

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [34]:
#mejores parametros
regressor_svr.best_params_

{'C': 21, 'gamma': 3, 'kernel': 'linear'}

In [35]:
#se realiza la prediccion para luego calular el mean_squared_error
ypred2 = regressor_svr.predict(xtest_scal)

In [36]:
#calculo el mean_squared_error.
np.sqrt(mean_squared_error(ytest, ypred2))

37.450465616125065

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [37]:
param_svreg = {'kernel':('linear', 'rbf'), 'C':[50, 5, 10 ], 'gamma':[ 10,1,30]}

In [38]:
regressor_svr = GridSearchCV(svreg, param_svreg, refit=True ,cv=5, verbose=True)

In [39]:
regressor_svr.fit(xtrain_scal, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [50, 5, 10], 'gamma': [10, 1, 30],
                         'kernel': ('linear', 'rbf')},
             verbose=True)

In [40]:
#mejores paramettros.
regressor_svr.best_params_

{'C': 50, 'gamma': 10, 'kernel': 'linear'}

In [41]:
#se realiza las predicciones
ypred3 = regressor_svr.predict(xtest_scal)

In [42]:
#calculalamos el mean_squared_error.
np.sqrt(mean_squared_error(ytest, ypred3))
#se observa una minima reduccion del mean_squared_error.

37.45027836558582